# Street car delay data preprocessing



In [21]:
# import cell
import os as os
import pandas as pd
import xlrd
import yaml
import pickle
from sklearn import preprocessing
import datetime

In [22]:
# Constant cell
root_directory = os.path.join(os.getcwd(), '../')
data_directory = os.path.join(root_directory ,'data')
data_raw_pickle_directory = os.path.join(data_directory, 'dataraw_pickle')
excel_name_list = []
for x in range(2014, 2019):
    excel_name = 'ttc-streetcar-delay-data-' + str(x) + '.xlsx'
    excel_name_list.append(excel_name)

with open('streetcar_data_preparation_config.yml', 'r') as config:
    try:
        config = yaml.safe_load(config)
        load_raw_pickle = config['general']['load_raw_pickle']
        load_cleaned_pickle = config['general']['load_cleaned_pickle']
    except yaml.YAMLError as e:
        print(e)
collist = ['Route', 'Day', 'Direction', 'Vehicle']
continuouscols = ['Min Gap', 'Min Delay']
textcols = ['Location', 'Incident']
data_cleaned_pickle_directory = os.path.join(data_directory, 'data_cleaned_pickle')
LableEncoder = preprocessing.LabelEncoder()
incident_encoder = preprocessing.OneHotEncoder(sparse=False)
day_encoder = preprocessing.OneHotEncoder(sparse=False)
direction_encoder = preprocessing.OneHotEncoder(sparse=False)


In [23]:
# valid data list
valid_day = ['Monday', 'Tuesdaty', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
valid_Direction = ['BW', 'NB', 'SB', 'WB', 'EB']
streetcar_vehicles = list(range(4000,4006))+ list(range(4010,4200)) +  list(range(4200,4252)) + [4900]
streetcar_vehicles = streetcar_vehicles + [4400] + list(range(4402,4508))
bus_vehicles = list(range(1000,1150))+ list(range(2000,2111)) + list(range(2150,2156)) + list(range(2240,2486))
bus_vehicles = bus_vehicles + list(range(2600,2620)) + list(range(2700,2766)) + list(range(2767,2859))
bus_vehicles = bus_vehicles + list(range(7000,7135)) + list(range(7400,7450)) + list(range(7500,7620)) + list(range(7620,7882))
bus_vehicles = bus_vehicles + list(range(8000,8100)) + list(range(9000,9027))
valid_vehicles = streetcar_vehicles + bus_vehicles
valid_vehicles = list(map(float, valid_vehicles))
valid_routes = [501,502,503,504,505,506,509,510,511,512,301,304,306,310]
valid_routes = list(map(str, valid_routes))
valid_vehicles = list(map(str, valid_vehicles))


In [44]:
#function cell
def excel2df(file_names):
    i = 1
    for f in file_names:
        file_directory = data_directory + '/' + f
        new_excel_file = pd.ExcelFile(file_directory)
        for sheet in new_excel_file.sheet_names:
            sheet_data = pd.read_excel(file_directory, sheet_name=sheet)
            if(i == 1):
                df = sheet_data
                i += 1
            else:
                df = pd.concat([df, sheet_data], ignore_index = True)
    return df

def save_pickle(df, name):
    df.to_pickle(data_directory + "/" + name)
    return df

# def drop_missing(dataset):
#     dataset.dropna()
#     return dataset


def check_valid(x, valid_list):
    if x in valid_list:
        return(x)
    else:
        return('bad value')

def remove_invalid_values(dataset, col, valid_list):
    dataset[col] = dataset[col].apply(lambda x: check_valid(x, valid_list))
    if_bad_value = (dataset[col] == 'bad value')
    drop_index = dataset.loc[if_bad_value].index
    drop_index = drop_index.tolist()
    drop_index.sort()
    drop_index = list(set(drop_index))
    dataset.drop(index=drop_index, axis = 0,inplace=True)
    return dataset

def direction_cleanup(dataset):
    dataset['Direction'].replace({'eastbound': 'EB', 'westbound': 'WB', 'southbound':'SB', 'northbound':'NB'}, inplace=True)
    dataset['Direction'] =dataset['Direction'].str.upper()
    dataset['Direction'].replace({'W/B' :'WB', 'E/B': 'EB', 'N/B': 'NB', 'S/B':'SB', 'B/W':'BW', 'B':'BW', 'b':'BW', 'W':'WB', 'E':'EB', 'N':'NB', 'S':'SB'}, inplace=True)
    return dataset   

def contain_and(x):
    return 'and' in x

def switch_and_phrase(x):
    new_pre_str = ''
    new_post_str = ''
    newx = ''
    post = False
    for str in x:
        if(str != 'and' and not post):
            new_pre_str += str
        elif(post):
            new_post_str += str
        else:
            post = True
            continue
    if(new_pre_str > new_post_str and contain_and(x)):
        newx = new_post_str + ' ' + 'and' + ' ' + new_pre_str
    elif(contain_and(x)):
        newx = new_pre_str + ' ' + 'and'  + ' ' + new_post_str
    else:
        for str in x:
            newx += str
            newx += ' '
    return newx
def clean_the_and_phrase(data):
    splited_location = data['Location'].str.split()
    new_location_representation = splited_location.apply(lambda x : switch_and_phrase(x))
    data['Location'] = new_location_representation
    return data

def location_cleanup(data):
    data['Location'] = data['Location'].str.replace('/', '')
    data['Location'] = data['Location'].str.replace('@', '')
    data['Location'] = data['Location'].str.lower()
    data = clean_the_and_phrase(data)
    return data

def one_hot_encoding(one_hot_encoder, dataset, col, columns):
    one_hot_encoder.fit(dataset[[col]])
    return pd.DataFrame(one_hot_encoder.transform(dataset[[col]]), columns=columns)

def parse_hour(x):
    return x.hour

def int_day_to_str(x):
    day_list = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday', 'Sunday']
    return day_list[x]

In [25]:
# load raw data
if(load_raw_pickle and not load_cleaned_pickle):
    df = pd.read_pickle(data_raw_pickle_directory)
elif(load_cleaned_pickle):
    df = pd.read_pickle(data_cleaned_pickle_directory)
else:
    df = excel2df(excel_name_list)
    save_pickle(df, "dataraw_pickle")
df = df.dropna()
df['Vehicle'] = df['Vehicle'].astype('str')
df['Route'] = df['Route'].astype('str')

In [26]:
# remove invalid data
direction_cleanup(df)
df = remove_invalid_values(df, 'Direction', valid_Direction)
print(df.shape[0])
remove_invalid_values(df, 'Vehicle',valid_vehicles)
print(df.shape[0])
df = remove_invalid_values(df, 'Route', valid_routes)
print(df.shape[0])

53871
53871
53871


In [27]:
# location clean up
df =location_cleanup(df)


In [28]:
save_pickle(df, "data_cleaned_pickle")

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle
0,2014-01-02,505,06:31:00,Thursday,dundas and roncesvalles,Late Leaving Garage,4.0,8.0,EB,4018.0
1,2014-01-02,504,12:43:00,Thursday,king and shaw,Utilized Off Route,20.0,22.0,EB,4128.0
2,2014-01-02,501,14:01:00,Thursday,bingham and kingstonroad,Held By,13.0,19.0,WB,4016.0
3,2014-01-02,504,14:22:00,Thursday,kingst. and roncesvallesave.,Investigation,7.0,11.0,WB,4175.0
4,2014-01-02,504,16:42:00,Thursday,bathurst and king,Utilized Off Route,3.0,6.0,EB,4080.0
...,...,...,...,...,...,...,...,...,...,...
66638,2018-12-31,504,21:53:00,Monday,berkeley,Held By,18.0,23.0,WB,4464.0
66639,2018-12-31,506,22:05:00,Monday,coxwell and uppergerard,Mechanical,8.0,14.0,EB,4150.0
66640,2018-12-31,501,22:09:00,Monday,neville loop,Mechanical,8.0,17.0,EB,4024.0
66641,2018-12-31,501,23:23:00,Monday,spadinalakeshore,Investigation,8.0,17.0,SB,4066.0


In [29]:
# Add derived columns
df['Year'] = pd.DatetimeIndex(df['Report Date']).year
df['Month'] = pd.DatetimeIndex(df['Report Date']).month
df['Daym'] = pd.DatetimeIndex(df['Report Date']).day

In [30]:
# Categorical data processing, Location

lableEncoder_items = df.Location.to_list()
LableEncoder.fit(lableEncoder_items)
location_lable = LableEncoder.transform(lableEncoder_items)
df['Location'] = location_lable


In [31]:
#Categorical data processing, Incident
#df.Incident = one_hot_encoding(incident_encoder, df, 'Incident')

In [32]:
#Categorical data processing, Day
#df.Day = one_hot_encoding(day_encoder, df, 'Day')

In [33]:
# Adjust time to continuous value
df['hour'] = df.Time.apply(lambda x: parse_hour(x))


In [34]:
# Minimizing data leakage
df = df.drop(columns=['Incident', 'Min Delay', 'Min Gap'])

In [35]:
df['target'] = 1
df = df.drop(columns='Time')

In [36]:
direction_df = pd.DataFrame({'Direction':valid_Direction, 'count':0})
route_df = pd.DataFrame({'Route': valid_routes, 'count':0})
direction_route_df = pd.merge(route_df, direction_df,on='count', how='outer')
hour_df = pd.DataFrame({'hour': range(0, 24), 'count':0})
direction_hour_route_df = pd.merge(direction_route_df, hour_df, on='count', how='outer')
date_list = pd.date_range(start='2014-01-02', end='2018-12-31').tolist()
date_df = pd.DataFrame({'Report Date':date_list, 'count':0})
date_df['Year'] = pd.DatetimeIndex(date_df['Report Date']).year
date_df['Month'] = pd.DatetimeIndex(date_df['Report Date']).month
date_df['Daym'] = pd.DatetimeIndex(date_df['Report Date']).day
date_df['Day'] = pd.DatetimeIndex(date_df['Report Date']).dayofweek
date_df['Day'] = date_df['Day'].apply(lambda x : int_day_to_str(x))
dhrd_df = pd.merge(direction_hour_route_df, date_df, on='count', how='outer')
dhrd_df = dhrd_df.drop(columns=['Year','Daym', 'Month', 'count', 'Day', 'count'])
merged_df = pd.merge(dhrd_df, df, on=['Report Date', 'Direction', 'hour', 'Route'], how='left')
merged_df['Year'] = pd.DatetimeIndex(merged_df['Report Date']).year
merged_df['Month'] = pd.DatetimeIndex(merged_df['Report Date']).month
merged_df['Daym'] = pd.DatetimeIndex(merged_df['Report Date']).day
merged_df['Day'] = pd.DatetimeIndex(merged_df['Report Date']).dayofweek
merged_df['Day'] = merged_df['Day'].apply(lambda x : int_day_to_str(x))
merged_df['Day'] = one_hot_encoding(day_encoder, merged_df, 'Day')
merged_df['target'] = merged_df['target'].fillna(0)

In [46]:

merged_df = pd.concat([merged_df, one_hot_encoding(direction_encoder, merged_df, 'Direction', columns=['dir1','dir2', 'dir3', 'dir4', 'dir5'])], axis = 1)
merged_df = pd.concat([merged_df, one_hot_encoding(day_encoder, merged_df, 'Day', columns=['wd1', 'wd2', 'wd3', 'wd4', 'wd5', 'wd6'])])

,Route,Direction,hour,Report Date,Day,Year,Month,Daym,target,dir1,dir2,dir3,dir4,dir5
0,501,BW,0,2014-01-02,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",2014,1,2,0.0,1.0,0.0,0.0,0.0,0.0
1,501,BW,0,2014-01-03,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2014,1,3,0.0,1.0,0.0,0.0,0.0,0.0
2,501,BW,0,2014-01-04,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",2014,1,4,0.0,1.0,0.0,0.0,0.0,0.0
3,501,BW,0,2014-01-05,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",2014,1,5,0.0,1.0,0.0,0.0,0.0,0.0
4,501,BW,0,2014-01-06,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2014,1,6,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3072885,310,EB,23,2018-12-27,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",2018,12,27,0.0,0.0,1.0,0.0,0.0,0.0
3072886,310,EB,23,2018-12-28,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2018,12,28,0.0,0.0,1.0,0.0,0.0,0.0
3072887,310,EB,23,2018-12-29,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",2018,12,29,0.0,0.0,1.0,0.0,0.0,0.0
3072888,310,EB,23,2018-12-30,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",2018,12,30,0.0,0.0,1.0,0.0,0.0,0.0


In [38]:
merged_df = merged_df.drop(columns=['Vehicle', 'Location'])

In [39]:
save_pickle(merged_df, "After_preprocessing_raw_pickle")

,Route,Direction,hour,Report Date,Day,Year,Month,Daym,target
0,501,BW,0,2014-01-02,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",2014,1,2,0.0
1,501,BW,0,2014-01-03,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2014,1,3,0.0
2,501,BW,0,2014-01-04,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",2014,1,4,0.0
3,501,BW,0,2014-01-05,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",2014,1,5,0.0
4,501,BW,0,2014-01-06,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2014,1,6,0.0
...,...,...,...,...,...,...,...,...,...
3072885,310,EB,23,2018-12-27,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",2018,12,27,0.0
3072886,310,EB,23,2018-12-28,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2018,12,28,0.0
3072887,310,EB,23,2018-12-29,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",2018,12,29,0.0
3072888,310,EB,23,2018-12-30,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",2018,12,30,0.0
